In [7]:
import numpy as np
import pandas as pd
import os 

In [8]:
aas_residualmz={"G":57.02147,"A":71.03712,"S":87.03203,"P":97.05277,"V":99.06842,"T":101.04768,"C":103.00919,"I":113.08407,"L":113.08407,"N":114.04293
,"D":115.02695,"Q":128.05858,"K":128.09497,"E":129.0426,"M":131.04049,"H":137.05891,"F":147.06842,"R":156.10112,"Y":163.06333,"W":186.07932}
aas=['A' ,'C', 'D' ,'E', 'F', 'G', 'H', 'I' ,'K', 'L', 'M', 'N' ,'P' ,'Q' ,'R' ,'S', 'T', 'V', 'W' ,'Y']

In [9]:
path="C:\\Workplace\\C#\\Test\\immonium_mass_neh\\immonium_mass\\bin\\x64\\Debug\\"

data=pd.DataFrame()
for f in ["Liver_l21.csv"]:    
    data=pd.concat([data,pd.read_csv(path+f)])
    
    
data["ion_count"]=[ len(str(x).split('|')) for x in data[["values"]].values]
print(data.shape)
data=data[data.ion_count>=2]
data=data.reset_index(drop=True)
print(data.shape)

(12179, 9)
(9284, 9)


In [10]:
all_mz=pd.DataFrame(np.unique([ int(float(j)*100)/100 for i in [x for x in [x.split("|") for x in data["values"]] ] for j in i], return_counts=True)).transpose()
all_mz.columns=["mz","freq"]
all_mz=all_mz[all_mz.freq>100]

In [11]:
dict_res={x:{"count":0,"total":0,"aas":[],"peps":[]} for x in all_mz.mz}
count=0
for i in range(data.shape[0]): 
    temp=np.unique([ int(float(x)*100)/100 for x in data["values"][i].split("|")])
    temp=[x for x in temp if x in list(all_mz.mz)]
    if len(temp)==0:
        continue
        
    for mz in all_mz[all_mz.mz>= min(temp)].mz:
        dict_res[mz]["total"]+=1        
    
    for mz in temp:
        dict_res[mz]["count"]+=1
        dict_res[mz]["aas"]+=list(np.unique(list(data["peptide"][i])))
        dict_res[mz]["peps"]+=[data["peptide"][i]]
    

In [16]:
dict_res[110.07]['count']/dict_res[110.07]['total']

0.8647363872082974

In [24]:
mz=110.07
df=pd.DataFrame(np.unique([x for x in dict_res[mz]["aas"] if x not in ['R','K','L']],return_counts=True)).transpose()
df.columns=['aa','perc']
df.perc=df.perc/dict_res[110.07]['count']
df=df.sort_values(by='perc',ascending=False)
df

,aa,perc
14,V,0.585374
0,A,0.553223
5,G,0.535399
12,S,0.505414
13,T,0.491921
3,E,0.486423
7,I,0.485591
2,D,0.465601
4,F,0.388806
10,P,0.388806


In [114]:
# selected_res=[]
# for k in dict_res.keys():
# #     print(k,dict_res[k]["count"],dict_res[k]["total"],"\t===>\t",100*dict_res[k]["count"]/dict_res[k]["total"])
#     selected_res.append([k,dict_res[k]["count"],dict_res[k]["total"],100*dict_res[k]["count"]/dict_res[k]["total"]])

In [115]:
# selected_res=pd.DataFrame(selected_res)
# selected_res.columns=["mz","freq","Total","perc"]
# selected_res=selected_res.sort_values(by="perc",ascending=False).reset_index(drop=True)
# selected_res.to_csv("possible_mz.csv",index=False)
# selected_res.head(50)

In [116]:
selected_res=[]
for k in dict_res.keys():
    
    possible_aa=list(set([key for key,val in aas_residualmz.items() if val >=k ])- set(['K','L','R']))
    temp=(pd.DataFrame(np.unique([x for x in dict_res[k]["aas"] if x in possible_aa],return_counts=True)).transpose())
    temp.columns=["aa","perc"]
    temp.perc=100*temp.perc/dict_res[k]["count"]
    temp=temp.sort_values(by="perc",ascending=False)
    temp=(temp[temp.perc>30].reset_index(drop=True))
    if (temp.shape[0]>0):
        print("mz:",k,"count:",f'{dict_res[k]["count"]}({int(1000*dict_res[k]["count"]/dict_res[k]["total"])/10})',"Range available:",dict_res[k]["total"],"\n")
        print(temp.head(9))
        print("------------------------------------------\n")
  

mz: 65.41 count: 127(100.0) Range available: 127 

  aa       perc
0  V  58.267717
1  A  54.330709
2  I  41.732283
3  F  35.433071
4  S  35.433071
5  P  33.070866
------------------------------------------

mz: 69.07 count: 312(80.8) Range available: 386 

  aa       perc
0  I  69.551282
1  A  49.038462
2  V  45.833333
3  F  38.141026
4  S  35.576923
5  T  31.730769
------------------------------------------

mz: 70.06 count: 887(98.5) Range available: 900 

  aa       perc
0  V  51.296505
1  A  50.958286
2  I  44.081172
3  F  38.782413
4  S  34.498309
5  T  32.919955
6  P   31.45434
7  D  30.101466
------------------------------------------

mz: 71.06 count: 180(19.6) Range available: 917 

  aa       perc
0  P  57.222222
1  I  51.111111
2  V  48.888889
3  F  37.222222
4  S  36.666667
5  T  31.666667
------------------------------------------

mz: 71.07 count: 273(29.5) Range available: 925 

  aa       perc
0  P  56.043956
1  V  53.846154
2  I  47.252747
3  S  38.461538
4  F  33.6996

mz: 119.08 count: 303(3.8) Range available: 7838 

  aa       perc
0  Q  56.105611
1  E  46.534653
2  F  37.293729
3  Y  32.013201
------------------------------------------

mz: 120.07 count: 508(6.4) Range available: 7880 

  aa       perc
0  F   75.19685
1  E   45.07874
2  Q  31.692913
------------------------------------------

mz: 120.08 count: 7490(88.0) Range available: 8502 

  aa       perc
0  E  50.280374
1  F   47.40988
2  Q  33.711615
3  Y  30.774366
------------------------------------------

mz: 121.08 count: 2490(29.1) Range available: 8549 

  aa       perc
0  F  67.188755
1  E  48.032129
------------------------------------------

mz: 122.07 count: 134(1.5) Range available: 8554 

  aa       perc
0  H  85.820896
1  E  50.746269
2  Q   33.58209
3  F  31.343284
------------------------------------------

mz: 125.07 count: 256(2.9) Range available: 8576 

  aa       perc
0  E   47.65625
1  F  43.359375
2  Q  37.890625
3  W   30.46875
4  Y  30.078125
----------------------

In [85]:
selected_res=[]
for k in dict_res.keys():
    
    possible_aa=list(set([key for key,val in aas_residualmz.items() if val >=k ])- set(['K','L','R']))
    
    temp=[]
    for i in range(len(possible_aa)):
        for j in range(i+1,len(possible_aa)):
            for pep in dict_res[k]["peps"]:
                if possible_aa[i] in pep and possible_aa[j] in pep:
                    temp.append("".join(sorted(possible_aa[i]+possible_aa[j])))               
    
    temp=(pd.DataFrame(np.unique(temp,return_counts=True)).transpose())
    temp.columns=["aa","perc"]
    temp.perc=100*temp.perc/dict_res[k]["count"]
    temp=temp.sort_values(by="perc",ascending=False)
    temp=(temp[temp.perc>50].reset_index(drop=True))
    if temp.shape[0]>0:
        print(k,dict_res[k]["count"],dict_res[k]["total"],"\t===>\t",100*dict_res[k]["count"]/dict_res[k]["total"],"\n")
        print(temp)
        print("------------------------------------------\n")
    
    
#     selected_res.append([k,dict_res[k]["count"],dict_res[k]["total"],100*dict_res[k]["count"]/dict_res[k]["total"]])

104.04 122 9187 	===>	 1.3279634265810385 

   aa       perc
0  EM  53.278689
------------------------------------------



In [16]:
[key for key,val in aas_residualmz.items() if val >=75 ]

['S',
 'P',
 'V',
 'T',
 'C',
 'I',
 'L',
 'N',
 'D',
 'Q',
 'K',
 'E',
 'M',
 'H',
 'F',
 'R',
 'Y',
 'W']